In [3]:
#决策树分类器
# 我们以iris数据集（iris）为例进行分析。iris以鸢尾花的特征作为数据来源，数据集包含150个数据集，分为3类，每类50个数据，每个数据包含4个属性
# ，是在数据挖掘、数据分类中非常常用的测试集、训练集。决策树可以用于分类和回归
from pyspark.ml.linalg import Vector,Vectors
from pyspark.sql import Row
from pyspark.ml import Pipeline
from pyspark.ml.feature import IndexToString,StringIndexer,VectorIndexer
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [13]:
#读取文本文件，第一个map把每行的数据用“,”隔开，比如在我们的数据集中，每行被分成了5部分，前4部分是鸢尾花的4个特征，最后一部分是鸢尾花的
#分类；我们这里把特征存储在Vector中，创建一个Iris模式的RDD，然后转化成dataframe；然后把刚刚得到的数据注册成一个表iris，注册成这个表
#之后，我们就可以通过sql语句进行数据查询
def f(x):
    rel = {}
    rel['features'] = Vectors.dense(float(x[0]),float(x[1]),float(x[2]),float(x[3]))
    rel['label'] = str(x[4])
    return rel
 
data = spark.sparkContext.textFile("file:///home/hadoop/iris.txt").map(lambda line: line.split(',')).map(lambda p: Row(**f(p))).toDF()
 
data.createOrReplaceTempView("iris")
 
df = spark.sql("select * from iris")
 
rel = df.rdd.map(lambda t : str(t[1])+":"+str(t[0])).collect()
for item in rel:
    print(item)

0:[5.1,3.5,1.4,0.2]
0:[4.9,3.0,1.4,0.2]
0:[4.7,3.2,1.3,0.2]
0:[4.6,3.1,1.5,0.2]
0:[5.0,3.6,1.4,0.2]
0:[5.4,3.9,1.7,0.4]
0:[4.6,3.4,1.4,0.3]
0:[5.0,3.4,1.5,0.2]
0:[4.4,2.9,1.4,0.2]
0:[4.9,3.1,1.5,0.1]
0:[5.4,3.7,1.5,0.2]
0:[4.8,3.4,1.6,0.2]
0:[4.8,3.0,1.4,0.1]
0:[4.3,3.0,1.1,0.1]
0:[5.8,4.0,1.2,0.2]
0:[5.7,4.4,1.5,0.4]
0:[5.4,3.9,1.3,0.4]
0:[5.1,3.5,1.4,0.3]
0:[5.7,3.8,1.7,0.3]
0:[5.1,3.8,1.5,0.3]
0:[5.4,3.4,1.7,0.2]
0:[5.1,3.7,1.5,0.4]
0:[4.6,3.6,1.0,0.2]
0:[5.1,3.3,1.7,0.5]
0:[4.8,3.4,1.9,0.2]
0:[5.0,3.0,1.6,0.2]
0:[5.0,3.4,1.6,0.4]
0:[5.2,3.5,1.5,0.2]
0:[5.2,3.4,1.4,0.2]
0:[4.7,3.2,1.6,0.2]
0:[4.8,3.1,1.6,0.2]
0:[5.4,3.4,1.5,0.4]
0:[5.2,4.1,1.5,0.1]
0:[5.5,4.2,1.4,0.2]
0:[4.9,3.1,1.5,0.1]
0:[5.0,3.2,1.2,0.2]
0:[5.5,3.5,1.3,0.2]
0:[4.9,3.1,1.5,0.1]
0:[4.4,3.0,1.3,0.2]
0:[5.1,3.4,1.5,0.2]
0:[5.0,3.5,1.3,0.3]
0:[4.5,2.3,1.3,0.3]
0:[4.4,3.2,1.3,0.2]
0:[5.0,3.5,1.6,0.6]
0:[5.1,3.8,1.9,0.4]
0:[4.8,3.0,1.4,0.3]
0:[5.1,3.8,1.6,0.2]
0:[4.6,3.2,1.4,0.2]
0:[5.3,3.7,1.5,0.2]
0:[5.0,3.3,1.4,0.2]


In [14]:
#分别获取标签列和特征列，进行索引，并进行了重命名。
labelIndexer = StringIndexer().setInputCol("label").setOutputCol("indexedLabel").fit(df)
 
featureIndexer = VectorIndexer().setInputCol("features").setOutputCol("indexedFeatures").setMaxCategories(4).fit(df)
 
#这里我们设置一个labelConverter，目的是把预测的类别重新转化成字符型的。
labelConverter = IndexToString().setInputCol("prediction").setOutputCol("predictedLabel").setLabels(labelIndexer.labels)
#接下来，我们把数据集随机分成训练集和测试集，其中训练集占70%。
trainingData, testData = data.randomSplit([0.7, 0.3])

In [15]:
#导入所需要的包
from pyspark.ml.classification import DecisionTreeClassificationModel,DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
#训练决策树模型,这里我们可以通过setter的方法来设置决策树的参数，也可以用ParamMap来设置（具体的可以查看spark mllib的官网）。具体的可以设置的参数可以通过explainParams()来获取。
dtClassifier = DecisionTreeClassifier().setLabelCol("indexedLabel").setFeaturesCol("indexedFeatures")
#在pipeline中进行设置
pipelinedClassifier = Pipeline().setStages([labelIndexer, featureIndexer, dtClassifier, labelConverter])
#训练决策树模型
modelClassifier = pipelinedClassifier.fit(trainingData)
#进行预测
predictionsClassifier = modelClassifier.transform(testData)
#查看部分预测的结果
predictionsClassifier.select("predictedLabel", "label", "features").show(20)

+--------------+-----+-----------------+
|predictedLabel|label|         features|
+--------------+-----+-----------------+
|             0|    0|[4.4,2.9,1.4,0.2]|
|             0|    0|[4.4,3.0,1.3,0.2]|
|             0|    0|[4.7,3.2,1.6,0.2]|
|             0|    0|[4.8,3.4,1.9,0.2]|
|             0|    0|[5.0,3.0,1.6,0.2]|
|             0|    0|[5.0,3.2,1.2,0.2]|
|             0|    0|[5.0,3.3,1.4,0.2]|
|             0|    0|[5.0,3.4,1.6,0.4]|
|             0|    0|[5.0,3.5,1.3,0.3]|
|             1|    1|[5.1,2.5,3.0,1.1]|
|             0|    0|[5.1,3.4,1.5,0.2]|
|             0|    0|[5.1,3.5,1.4,0.3]|
|             0|    0|[5.1,3.7,1.5,0.4]|
|             0|    0|[5.1,3.8,1.5,0.3]|
|             0|    0|[5.2,3.4,1.4,0.2]|
|             0|    0|[5.4,3.4,1.7,0.2]|
|             1|    1|[5.5,2.5,4.0,1.3]|
|             2|    2|[5.6,2.8,4.9,2.0]|
|             0|    0|[5.7,3.8,1.7,0.3]|
|             1|    1|[5.8,2.6,4.0,1.2]|
+--------------+-----+-----------------+
only showing top

In [27]:
#评估决策树分类模型
evaluatorClassifier = MulticlassClassificationEvaluator().setLabelCol("indexedLabel").setPredictionCol("prediction").setMetricName("accuracy")
 
accuracy = evaluatorClassifier.evaluate(predictionsClassifier)
 
print("Test Error = " + str(1.0 - accuracy))
#Test Error = 0.05882352941176472
 
treeModelClassifier = modelClassifier.stages[2]
 
print("Learned classification tree model:\n" + str(treeModelClassifier.toDebugString))

Test Error = 0.08510638297872342
Learned classification tree model:
DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4aa0a79c6d987d71bfeb) of depth 5 with 13 nodes
  If (feature 2 <= 1.9)
   Predict: 0.0
  Else (feature 2 > 1.9)
   If (feature 2 <= 4.7)
    If (feature 3 <= 1.6)
     Predict: 2.0
    Else (feature 3 > 1.6)
     Predict: 1.0
   Else (feature 2 > 4.7)
    If (feature 2 <= 5.0)
     If (feature 1 <= 2.8)
      Predict: 1.0
     Else (feature 1 > 2.8)
      If (feature 0 <= 5.9)
       Predict: 2.0
      Else (feature 0 > 5.9)
       Predict: 1.0
    Else (feature 2 > 5.0)
     Predict: 1.0



In [20]:
#构建决策树回归模型
#导入所需要的包
from pyspark.ml.regression import DecisionTreeRegressionModel,DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
#训练决策树模型
dtRegressor = DecisionTreeRegressor().setLabelCol("indexedLabel").setFeaturesCol("indexedFeatures")
#在pipeline中进行设置
pipelineRegressor = Pipeline().setStages([labelIndexer, featureIndexer, dtRegressor, labelConverter])
#训练决策树模型
modelRegressor = pipelineRegressor.fit(trainingData)
#进行预测
predictionsRegressor = modelRegressor.transform(testData)
#查看部分预测结果
predictionsRegressor.select("predictedLabel", "label", "features").show(20)

+--------------+-----+-----------------+
|predictedLabel|label|         features|
+--------------+-----+-----------------+
|             0|    0|[4.4,2.9,1.4,0.2]|
|             0|    0|[4.4,3.0,1.3,0.2]|
|             0|    0|[4.7,3.2,1.6,0.2]|
|             0|    0|[4.8,3.4,1.9,0.2]|
|             0|    0|[5.0,3.0,1.6,0.2]|
|             0|    0|[5.0,3.2,1.2,0.2]|
|             0|    0|[5.0,3.3,1.4,0.2]|
|             0|    0|[5.0,3.4,1.6,0.4]|
|             0|    0|[5.0,3.5,1.3,0.3]|
|             1|    1|[5.1,2.5,3.0,1.1]|
|             0|    0|[5.1,3.4,1.5,0.2]|
|             0|    0|[5.1,3.5,1.4,0.3]|
|             0|    0|[5.1,3.7,1.5,0.4]|
|             0|    0|[5.1,3.8,1.5,0.3]|
|             0|    0|[5.2,3.4,1.4,0.2]|
|             0|    0|[5.4,3.4,1.7,0.2]|
|             1|    1|[5.5,2.5,4.0,1.3]|
|             2|    2|[5.6,2.8,4.9,2.0]|
|             0|    0|[5.7,3.8,1.7,0.3]|
|             1|    1|[5.8,2.6,4.0,1.2]|
+--------------+-----+-----------------+
only showing top

In [26]:
evaluatorRegressor = RegressionEvaluator().setLabelCol("indexedLabel").setPredictionCol("prediction").setMetricName("rmse")
 
rmse = evaluatorRegressor.evaluate(predictionsRegressor)
 
print("Root Mean Squared Error (RMSE) on test data = " +str(rmse))
#Root Mean Squared Error (RMSE) on test data = 0.24253562503633297
 
treeModelRegressor = modelRegressor.stages[2]
 
print("Learned regression tree model:\n" + str(treeModelRegressor.toDebugString))

Root Mean Squared Error (RMSE) on test data = 0.27288841145490766
Learned regression tree model:
DecisionTreeRegressionModel (uid=DecisionTreeRegressor_4e5f821ce44f8a1f0818) of depth 5 with 13 nodes
  If (feature 2 <= 1.9)
   Predict: 0.0
  Else (feature 2 > 1.9)
   If (feature 2 <= 4.7)
    If (feature 3 <= 1.6)
     Predict: 2.0
    Else (feature 3 > 1.6)
     Predict: 1.0
   Else (feature 2 > 4.7)
    If (feature 2 <= 5.0)
     If (feature 1 <= 2.8)
      Predict: 1.0
     Else (feature 1 > 2.8)
      If (feature 0 <= 5.9)
       Predict: 2.0
      Else (feature 0 > 5.9)
       Predict: 1.5
    Else (feature 2 > 5.0)
     Predict: 1.0

